In [221]:
import tensorflow as tf
import numpy as np
import math

percent_test_samples = 10

# load file
dataset2014 = np.loadtxt(open("../../datasets/usage2014.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))
dataset2015 = np.loadtxt(open("../../datasets/usage2015.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))
dataset2016 = np.loadtxt(open("../../datasets/usage2016.csv", "rb"), delimiter=",", skiprows=1,  usecols=range(1,24))
# train_data = result[:8,:3]
# train_label = result[:8,3].reshape(8,1)
# test_data = result[[8,9],:3]
# test_label = result[[8,9],3].reshape(2,1)

# print(train_data.shape)
# print(train_label.reshape(8,1).shape)
# print(test_data.shape)
# print(test_label.reshape(2,1).shape)

# by features we mean how many columns will act as features and how many as labels
def get_training_features_indexes(dataset):
    rows_count, columns_count = dataset.shape
    print(columns_count)
    # features index, labels index
    return columns_count - 2, [columns_count - 1,columns_count - 2]
    

# by samples we mean how many rows will act as training samples and how many as test
def get_training_samples_indexes(dataset):
    rows_count, columns_count = dataset.shape
    # print(rows_count)
    test_samples_count = (rows_count * percent_test_samples / 100)

    # selecting the top n rows as training samples
    return math.ceil(rows_count - test_samples_count)
    
    


def get_features(dataset):

    first_n_training_samples_count = get_training_samples_indexes(dataset)
    features_columns, label_columns = get_training_features_indexes(dataset)

    training_features = dataset[:first_n_training_samples_count,:features_columns]
    training_labels = dataset[:first_n_training_samples_count,label_columns]


    test_features = dataset[first_n_training_samples_count:,:features_columns ]
    test_labels = dataset[first_n_training_samples_count:,label_columns ]
    
    return training_features, training_labels, test_features, test_labels




training_features_2014 , training_labels_2014, test_features_2014, test_labels_2014 = get_features(dataset2014)
training_features_2015 , training_labels_2015, test_features_2015, test_labels_2015 = get_features(dataset2015)
training_features_2016 , training_labels_2016, test_features_2016, test_labels_2016 = get_features(dataset2016)

training_features = np.concatenate((np.concatenate((training_features_2014,training_features_2015),axis=0),training_features_2016),axis=0)
training_labels = np.concatenate((np.concatenate((training_labels_2014,training_labels_2015),axis=0),training_labels_2016),axis=0)
test_features = np.concatenate((np.concatenate((test_features_2014,test_features_2015),axis=0),test_features_2016),axis=0)
test_labels = np.concatenate((np.concatenate((test_labels_2014,test_labels_2015),axis=0),test_labels_2016),axis=0)


print(training_features.shape)
print(training_labels.shape)
print(test_features.shape)
print(test_labels.shape)

23
23
23
(514789, 21)
(514789, 2)
(57198, 21)
(57198, 2)


In [ ]:

input_neurons = training_features.shape[1]
hl1_neuron = 100
hl2_neuron = 10
output_neurons = training_labels.shape[1]
batch_size = 100

x = tf.placeholder(tf.float32, [None,input_neurons])
y = tf.placeholder(tf.float32,[None,output_neurons])

hidden1_weights_biases = { "weights": tf.Variable(tf.truncated_normal([input_neurons,hl1_neuron])) , 
                         "biases": tf.Variable(tf.truncated_normal([hl1_neuron]))}

# hidden2_weights_biases = { "weights": tf.Variable(tf.truncated_normal([hl1_neuron,hl2_neuron])) , 
#                          "biases": tf.Variable(tf.truncated_normal([hl2_neuron]))}

output_weights_biases =  { "weights": tf.Variable(tf.truncated_normal([hl1_neuron,output_neurons])) , 
                         "biases": tf.Variable(tf.truncated_normal([output_neurons]))}

l1 = tf.add(tf.matmul(x,hidden1_weights_biases["weights"]), hidden1_weights_biases["biases"])
l1 = tf.nn.relu(l1)

# l2 = tf.add(tf.matmul(l1,hidden2_weights_biases["weights"]), hidden2_weights_biases["biases"])
# l2 = tf.nn.relu(l2)

prediction = tf.add(tf.matmul(l1,output_weights_biases["weights"]), output_weights_biases["biases"])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

hm_epochs = 10

def next_batch(batch_size):
    
    rows_from = batch_count
    rows_to = (batch_count+batch_size)
    # select row from 100 to 200 where 100 is the batch_Count and 200 is the batch_count + batch_size
    return training_features[rows_from:rows_to,:] , training_labels[rows_from:rows_to,:]
    
for epoch in range(hm_epochs):
    epoch_loss = 0
    batch_count = 0
    for index in range(int(training_features.shape[0]/batch_size)):
        batch_xs, batch_ys = next_batch(batch_size)
        batch_count+=batch_size
        _, c = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
        epoch_loss += c
    print('Epoch',epoch,'completed out of',hm_epochs,'loss:',epoch_loss)
    

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_features, y: test_labels}))


Epoch 0 completed out of 10 loss: 39387.546237
Epoch 1 completed out of 10 loss: 31964.9918781
Epoch 2 completed out of 10 loss: 24602.4398998
Epoch 3 completed out of 10 loss: 25937.4369233
Epoch 4 completed out of 10 loss: 28177.8821784
Epoch 5 completed out of 10 loss: 30611.6824739
